In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/stations-and-data.csv')
df.head()

,Unnamed: 0_x,elevation,mindate,maxdate,latitude,name,datacoverage,station,elevationUnit,longitude,...,TMIN-AVG-11,TMIN-AVG-12,TMIN-AVG-2,TMIN-AVG-3,TMIN-AVG-4,TMIN-AVG-5,TMIN-AVG-6,TMIN-AVG-7,TMIN-AVG-8,TMIN-AVG-9
0,0,89.6,1942-03-01,2020-01-26,32.3347,"MERIDIAN KEY FIELD, MS US",1.0,GHCND:USW00013865,METERS,-88.7442,...,42.100,41.025,45.275,46.875,52.200,62.100,70.325,72.950,72.875,68.750
1,0,89.6,1942-03-01,2020-01-26,32.3347,"MERIDIAN KEY FIELD, MS US",1.0,GHCND:USW00013865,METERS,-88.7442,...,42.100,41.025,45.275,46.875,52.200,62.100,70.325,72.950,72.875,68.750
2,0,160.0,2008-12-01,2020-01-25,33.5721,"TALLADEGA 10 NNE, AL US",1.0,GHCND:USW00073803,METERS,-86.0573,...,38.475,37.475,42.550,42.825,49.275,59.425,67.000,70.225,69.600,64.925
3,0,119.5,1948-01-01,2020-01-25,32.5161,"COLUMBUS METROPOLITAN AIRPORT, GA US",1.0,GHCND:USW00093842,METERS,-84.9422,...,45.575,43.275,47.025,47.900,54.325,64.500,70.975,73.550,73.325,70.025
4,0,53.6,2002-08-01,2020-01-25,31.3127,"NEWTON 8 W, GA US",1.0,GHCND:USW00063828,METERS,-84.4706,...,43.600,43.025,45.825,46.450,52.125,61.525,68.825,70.625,70.775,66.450


In [89]:
import requests

In [2]:
df.shape

(686, 71)

In [15]:
#!pip install plotly

import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = df.longitude.values.tolist(),
    lat = df.latitude.values.tolist(),
    marker = {'size': 11, 'color':'red'}))

fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lon = df.longitude.values.tolist(),
    lat = df.latitude.values.tolist(),
    marker = {'size': 4, 'color':'white'}))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 10, 'lat': 10},
        'style': "stamen-terrain",
        'center': {'lon': -20, 'lat': -20},
        'zoom': 1})

fig.show()

In [49]:
prcp = []
snow = []
tavg = []
tmax = []
tmin = []
for i in df.columns.tolist()[11:]:
    cat = i.split('-')[0]
    if cat == 'PRCP':
        prcp.append(i)
    elif cat == 'SNOW':
        snow.append(i)
    elif cat == 'TAVG':
        tavg.append(i)
    elif cat == 'TMAX':
        tmax.append(i)
    else:
        tmin.append(i)

In [50]:
for a in [prcp, snow, tavg, tmax, tmin]:
    name = a[0].split('-')[0].lower()
    mylist = a
    myorder = [0, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3]
    vars()[name] = [mylist[i] for i in myorder]


In [88]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

In [93]:
dflat = df.latitude.values.tolist()
dflon = df.longitude.values.tolist()

In [ ]:
rNuKKxAX3UL59FIt

In [102]:
!pip install pymongo[srv]

import pymongo

mongokey =#
client = pymongo.MongoClient(mongokey)
db = client.citydata
table = db.alldata
print('connected to mongodb')

connected to mongodb


In [106]:
mongokey2 = #
client2 = pymongo.MongoClient(mongokey2)
db2 = client2.citydata
table2 = db2.alldata
print('connected to mongodb')

connected to mongodb


In [114]:
for i in range(29000):
    get = table.find_one({'_id':i})
    lat = get['Latitude']
    lon = get['Longitude']
    dist_li = []
    for loc in range(len(dflon)):
        dist_val = distance(lat, lon, dflat[loc], dflon[loc])
        dist_li.append(dist_val)
    index_min = np.argmin(dist_li)
    
    #print(f"{lat} - {lon} | {df[['latitude']].values[index_min]} - {df[['longitude']].values[index_min]}")
    
    
    tempjsn = {}
    for a in [prcp, snow, tavg, tmax, tmin]:
        cat = a[0].split('-')[0].lower()
        dikeys = a
        divals = df[a].values[index_min].tolist()
        divals_fin = []
        for b in divals:
            if str(b) == 'nan':
                divals_fin.append(None)
            else:
                divals_fin.append(b)
        weatherdi = dict(zip(dikeys, divals_fin))
        tempjsn[cat] = weatherdi
    get['weather'] = tempjsn
    table2.insert_one(get)
 

TypeError: 'NoneType' object is not subscriptable